In [15]:
# import library you use
import pandas as pd

# Training phase

### Load Data

In [16]:
# load train.csv and train.csv
train = pd.read_csv("train.csv")
train_label = pd.read_csv("train_label.csv")
train

,mac_hash,sniffer_loc,created_time
0,10f6ad2e3fa3b3da8ceb3fbd95658681,2,2018-12-08 11:47:48
1,10f6ad2e3fa3b3da8ceb3fbd95658681,10,2018-12-08 12:37:34
2,10f6ad2e3fa3b3da8ceb3fbd95658681,9,2018-12-08 12:37:35
3,10f6ad2e3fa3b3da8ceb3fbd95658681,6,2018-12-08 12:44:48
4,10f6ad2e3fa3b3da8ceb3fbd95658681,4,2018-12-08 12:44:54
...,...,...,...
33222,13a3e10f950d8050ab5437cf30a3c2c6,4,2018-12-06 20:13:40
33223,13a3e10f950d8050ab5437cf30a3c2c6,9,2018-12-06 20:14:42
33224,13a3e10f950d8050ab5437cf30a3c2c6,3,2018-12-06 20:16:42
33225,13a3e10f950d8050ab5437cf30a3c2c6,1,2018-12-06 20:18:41


### Data Preprocessing

In [17]:
# drop created_time column
train = train.drop(['created_time'], axis=1)
train

,mac_hash,sniffer_loc
0,10f6ad2e3fa3b3da8ceb3fbd95658681,2
1,10f6ad2e3fa3b3da8ceb3fbd95658681,10
2,10f6ad2e3fa3b3da8ceb3fbd95658681,9
3,10f6ad2e3fa3b3da8ceb3fbd95658681,6
4,10f6ad2e3fa3b3da8ceb3fbd95658681,4
...,...,...
33222,13a3e10f950d8050ab5437cf30a3c2c6,4
33223,13a3e10f950d8050ab5437cf30a3c2c6,9
33224,13a3e10f950d8050ab5437cf30a3c2c6,3
33225,13a3e10f950d8050ab5437cf30a3c2c6,1


In [18]:
# Count whether each mac_hash has passed through these 14 locations, and generate a training dataframe
df = pd.DataFrame(train)
one_hot_loc = pd.get_dummies(df.loc[:,'sniffer_loc'])
df = df.drop(['sniffer_loc'], axis=1)
df = df.join(one_hot_loc)
mac = df.groupby('mac_hash', as_index=False).sum()
mac.head()

,mac_hash,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0003ae8541d0e925fcee242287e2ad27,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,00078611037990f7f36b722f22595fe7,0,1,1,1,1,1,0,1,0,1,1,0,0,0
2,000ce31739b333d931813de403156844,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,0027240b2aeb1f7f3c053c073cafb452,0,1,0,1,1,1,1,1,1,1,1,1,1,1
4,0028a89964416dc9a8a66cc7fe25ef4e,0,1,1,1,0,1,0,0,1,1,1,1,1,1


In [19]:
# Add label column
df1 = pd.DataFrame(train_label)
data_train = pd.merge(mac, df1)
data_train.head()

,mac_hash,1,2,3,4,5,6,7,8,9,10,11,12,13,14,label
0,0003ae8541d0e925fcee242287e2ad27,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,00078611037990f7f36b722f22595fe7,0,1,1,1,1,1,0,1,0,1,1,0,0,0,1
2,000ce31739b333d931813de403156844,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3
3,0027240b2aeb1f7f3c053c073cafb452,0,1,0,1,1,1,1,1,1,1,1,1,1,1,4
4,0028a89964416dc9a8a66cc7fe25ef4e,0,1,1,1,0,1,0,0,1,1,1,1,1,1,4


### Training

In [36]:
# X: feature from training dataframe
X = data_train[data_train.columns[1:15]]

In [37]:
# Y: label from training dataframe
# you need to change your label from categorical encoding to one-hot encoding
Y = data_train[data_train.columns[15]]
Y = pd.get_dummies(Y)
Y

,0,1,2,3,4
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
5563,0,0,0,0,1
5564,1,0,0,0,0
5565,0,0,0,0,1
5566,1,0,0,0,0


In [38]:
# split your dataset as training dataset and validation dataset
# using train_test_split function,
### training dataset : validation dataset = 8 : 2
### random_state = 0
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, train_size=0.8)
print(X_train)
print(X_test)


      1  2  3  4  5  6  7  8  9  10  11  12  13  14
253   0  1  1  1  1  1  1  1  1   1   1   1   1   0
1114  0  1  0  1  1  1  1  1  1   1   1   1   1   1
4257  0  1  1  1  1  1  0  0  1   1   1   1   1   0
3979  1  1  1  0  0  1  0  0  0   0   0   0   0   0
1012  0  1  0  0  1  0  0  0  0   0   0   1   0   0
...  .. .. .. .. .. .. .. .. ..  ..  ..  ..  ..  ..
4931  0  0  0  0  0  0  0  0  0   0   0   1   0   1
3264  0  1  1  1  1  1  1  1  1   0   1   1   1   0
1653  1  1  1  1  0  1  1  1  1   1   1   1   1   1
2607  0  0  0  1  0  1  0  0  0   0   0   0   0   0
2732  0  1  1  1  1  1  0  1  1   1   1   1   1   1

[4454 rows x 14 columns]
      1  2  3  4  5  6  7  8  9  10  11  12  13  14
3987  0  1  1  1  1  1  1  1  1   1   1   1   1   1
690   1  1  1  1  1  1  0  1  0   1   1   0   0   0
944   1  1  1  1  1  1  0  1  0   0   0   1   0   0
2565  0  1  1  0  0  0  0  0  0   0   0   1   0   1
4029  0  0  0  0  0  1  0  0  0   0   0   0   0   0
...  .. .. .. .. .. .. .. .. ..  ..  .

In [39]:
# build DNN model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

model = Sequential()
# 加入 hidden layer-1 of 512 neurons 並指定 input_dim 為 784
model.add(Dense(40, input_dim=14))
# 使用 'relu' 當作 activation function
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(70))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(21))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 40)                600       
                                                                 
 activation_12 (Activation)  (None, 40)                0         
                                                                 
 dropout_9 (Dropout)         (None, 40)                0         
                                                                 
 dense_13 (Dense)            (None, 70)                2870      
                                                                 
 activation_13 (Activation)  (None, 70)                0         
                                                                 
 dropout_10 (Dropout)        (None, 70)                0         
                                                                 
 dense_14 (Dense)            (None, 21)               

In [40]:
# start train model
# 定義訓練方式  
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
# validate your model using validation dataset
from sklearn import preprocessing
# 開始訓練  
train_history = model.fit(x=X_train, y=Y_train, validation_split=0.1, epochs=10, batch_size=40)  
# After prediction, you need to change your result and label from one-hot encoding to categorical encoding
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaledFeatures = minmax_scale.fit_transform(X_train)

score = model.evaluate(x=scaledFeatures, y=Y_train)
print ('\nTrain Loss:', score[0])
print ('\nTrain Acc:', score[1])

Epoch 1/10
101/101 [==============================] - 2s 6ms/step - loss: 1.3775 - accuracy: 0.4828 - val_loss: 0.9389 - val_accuracy: 0.6659
Epoch 2/10
101/101 [==============================] - 0s 3ms/step - loss: 0.8293 - accuracy: 0.6846 - val_loss: 0.4390 - val_accuracy: 0.8857
Epoch 3/10
101/101 [==============================] - 0s 3ms/step - loss: 0.5775 - accuracy: 0.7832 - val_loss: 0.2892 - val_accuracy: 0.9170
Epoch 4/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4772 - accuracy: 0.8276 - val_loss: 0.2321 - val_accuracy: 0.9260
Epoch 5/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3968 - accuracy: 0.8600 - val_loss: 0.1911 - val_accuracy: 0.9439
Epoch 6/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3753 - accuracy: 0.8718 - val_loss: 0.1811 - val_accuracy: 0.9529
Epoch 7/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3303 - accuracy: 0.8845 - val_loss: 0.1709 - val_accuracy: 0.9574
Epoch 

In [55]:
# using sklearn classification_report function to show your validation result
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
onehotencoder = OneHotEncoder()

# pred = np.argmax(model.predict(X_train), axis=-1)
# pred
predicted = onehotencoder.inverse_transform(model.predict(X_train))
predicted
# print(classification_report(pred, exp))

140/140 [==============================] - 0s 2ms/step


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Testing phase

### Load Data

In [ ]:
# load test.csv file as dataframe
test = pd.read_csv('test.csv')


### Data Preprocessing

In [ ]:
# Preprocessing is the same as training
data_test = pd.DataFrame(test)
data_test = data_test.drop(['created_time'], axis=1)
df = pd.DataFrame(test)
one_hot_loc = pd.get_dummies(df.loc[:,'sniffer_loc']).rename(columns={0:'1',1:'2',2:'3',3:'4',4:'5',5:'6',6:'7',7:'8',8:'9',9:'10',10:'11',11:'12',12:'13',13:'14'})
df = df.drop(['sniffer_loc'], axis=1)
df = df.join(one_hot_loc)
mac = df.groupby('mac_hash', as_index=False).sum()
mac.head()


C:\Users\user\AppData\Local\Temp\ipykernel_15280\2719597273.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mac = df.groupby('mac_hash', as_index=False).sum()


,mac_hash,2,3,4,5,6,7,8,9,10,11,12,13,14,14
0,004c76bfe9f7b20f3651e0bcbd8eee9e,1,1,1,1,1,1,0,1,1,0,0,0,0,0
1,0084303d531091816387956ea26b86eb,0,1,1,1,1,0,1,1,1,1,0,0,0,0
2,00872a1ee635c2619454350b96b94103,0,1,1,1,1,1,0,1,1,1,1,1,1,1
3,00a5f69fc7719ba29c7b202f292dbafb,0,0,1,1,1,1,1,1,1,1,1,1,1,1
4,00ad41313f1f637e079c1a9d9a857d12,0,1,1,0,1,1,0,1,1,0,1,1,1,0


### Predict the result using your model

In [ ]:
# X_test: feature

: 

In [ ]:
# Prediction

: 

In [ ]:
# Generate submit.csv

: 